In [1]:
import pandas as pd

In [17]:
df = pd.read_csv("../data/dataset-RedFlags.csv")
df

,Unnamed: 0,ocid,id,initiationType,language,date,tag,title,description,status,...,amount,dateSigned,id_awards,date_awards,amount_awards,currency,budget_id,budget_amount,ruc,Provincia
0,0,ocds-5wno2w-REGES-MPORTO-2021-13-2332,REGES-MPORTO-2021-13-2332-2022-05-07T11:56:24....,tender,es,2022-05-07T07:11:05-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",REGES-MPORTO-2021-13-2332,ESPACIOS DE DIFUSIÓN MASIVA A TRAVÉS DE VALLAS...,complete,...,13392.86,2021-05-31T00:00:00-05:00,4748528-REGES-MPORTO-2021-13,2021-05-07T14:21:51-05:00,13392.86,USD,2021.19.01.A100.150.530207.000.13.01.000.0.000,13392.86,1360000200001,Manabi
1,1,ocds-5wno2w-RE-EEASA-142-2021-3233,RE-EEASA-142-2021-3233-2022-05-10T11:35:00.178...,tender,es,2022-05-10T06:46:51-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",RE-EEASA-142-2021-3233,?SERVICIO DE PAUTAJE EN EL MEDIO DE COMUNICACI...,complete,...,7000.00,2021-10-20T00:00:00-05:00,5104157-RE-EEASA-142-2021,2021-10-15T08:28:28-05:00,7000.00,USD,52202030102120128,7000.00,1890001439001,Tungurahua
2,2,ocds-5wno2w-LICS-UNEMI-013-2021-43664,LICS-UNEMI-013-2021-43664-2022-05-11T11:32:30....,tender,es,2022-05-11T06:48:06-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",LICS-UNEMI-013-2021-43664,CONTRATACION DE POLIZAS DE SEGUROMULTIRIESGO: ...,complete,...,59510.25,2021-06-23T00:00:00-05:00,4720310-LICS-UNEMI-013-2021,2021-04-23T10:25:08-05:00,59510.25,USD,2021.181.0000.0000.82.00.000.002.000.0910.5303...,70005.18,968533430001,Guayas
3,3,ocds-5wno2w-RE-EP-GMJ-DGF-1-2021-102953,RE-EP-GMJ-DGF-1-2021-102953-2022-05-11T11:34:5...,tender,es,2022-05-11T06:48:07-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",RE-EP-GMJ-DGF-1-2021-102953,Elaboración y Adquisición de especiesvaloradas...,complete,...,1700.00,2021-08-02T00:00:00-05:00,4930830-RE-EP-GMJ-DGF-1-2021,2021-07-20T13:49:50-05:00,1700.00,USD,5.3.08.07,1700.00,1360000630001,Manabi
4,4,ocds-5wno2w-LIC-EPFLP-01-2021-404551,LIC-EPFLP-01-2021-404551-2022-05-14T12:25:47.8...,tender,es,2022-05-14T07:33:35-05:00,"[""planning"",""tender"",""award"",""contract""]",LIC-EPFLP-01-2021-404551,CONTRATACIÓN DE LA PÓLIZA DE SEGUROS MARÍTIMOS...,complete,...,749885.00,2021-09-15T00:00:00-05:00,4980937-LIC-EPFLP-01-2021,2021-08-30T08:43:16-05:00,749885.00,USD,5.1.2.02.01,910648.25,1768164300001,Pichincha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84696,84696,ocds-5wno2w-RE-EPMAPAS-001-2021-279324,RE-EPMAPAS-001-2021-279324-2022-04-28T11:33:29...,tender,es,2022-04-28T06:45:59-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",RE-EPMAPAS-001-2021-279324,CONTRATACION DE PAUTAS COMERCIALES EN EL PROGR...,complete,...,2400.00,2021-08-23T00:00:00-05:00,4992062-RE-EPMAPAS-001-2021,2021-08-18T10:04:49-05:00,2400.00,USD,6.3.02.07,2400.00,968586470001,Guayas
84697,84697,ocds-5wno2w-RE-SC-MDMQ-08-2021-41357,RE-SC-MDMQ-08-2021-41357-2022-04-28T11:30:12.1...,tender,es,2022-04-28T06:45:59-05:00,"[""planning"",""tender"",""award"",""contract"",""imple...",RE-SC-MDMQ-08-2021-41357,CONTRATACIÓN DEL SERVICIO COORDINACIÓN Y LOCUC...,complete,...,15600.00,2021-02-18T00:00:00-05:00,4661462-RE-SC-MDMQ-08-2021,2021-02-10T11:27:11-05:00,15.60,USD,730222,15.60,1760003410001,Pichincha
84698,84698,ocds-5wno2w-REOALC-GADMANTA01-21-2668,REOALC-GADMANTA01-21-2668-2022-04-30T12:21:31....,tender,es,2022-04-30T07:28:18-05:00,"[""planning"",""tender"",""award"",""contract""]",REOALC-GADMANTA01-21-2668,"CONTRATACIÓN DEL SERVICIO DE ORGANIZACIÓN, PRO...",complete,...,268000.00,2021-12-22T00:00:00-05:00,5151471-REOALC-GADMANTA01-21,2021-10-28T16:50:25-05:00,268000.00,USD,530205,268000.00,1360000980001,Manabi
84699,84699,ocds-5wno2w-RE-GADMCN-17-21-LRG-67188,RE-GADMCN-17-21-LRG-67188-2022-05-04T11:48:52....,tender,es,2022-05-04T06:51:48-05:00,"[""planning"",""tender"",""award"",""contract""]",RE-GADMCN-17-21-LRG-67188,PRESTACIÓN DE SERVICIOS DE INSPECCIÓN Y MANTEN...,active,...,30590.89,2022-0

In [18]:
df.dtypes

Unnamed: 0                         int64
ocid                              object
id                                object
initiationType                    object
language                          object
date                              object
tag                               object
title                             object
description                       object
status                            object
procuringEntity_id                object
procuringEntity_name              object
value_amount                     float64
value_currency                    object
procurementMethod                 object
procurementMethodDetails          object
numberOfTenderers                float64
tenderPeriod_durationInDays      float64
id_supplier                       object
name                              object
id_contracts                      object
status_contracts                  object
contractPeriod_durationInDays    float64
amount                           float64
dateSigned      

In [19]:
licitacionPorEntidad = df.groupby(['procuringEntity_name']).agg({'tenderPeriod_durationInDays': ['mean', 'min', 'max','size']}).reset_index()
licitacionPorEntidad

procuringEntity_name  \
                                                          
0     ACERIAS NACIONALES DEL ECUADOR SOCIEDAD ANONIM...   
1     AGENCIA DE ASEGURAMIENTO DE LA CALIDAD DE LOS ...   
2     AGENCIA DE REGULACION Y CONTROL DE LAS TELECOM...   
3     AGENCIA DE REGULACIÓN Y CONTROL DE ENERGÍA Y R...   
4              AGENCIA DE REGULACIÓN Y CONTROL DEL AGUA   
...                                                 ...   
1083                     Universidad Tecnica de Machala   
1084                      Universidad Técnica de Manabí   
1085                      Universidad Técnica del Norte   
1086                   VICEPRESIDENCIA DE LA REPUBLICA    
1087  comando provincial de la policia del guayas nr...   

     tenderPeriod_durationInDays                  
                            mean  min   max size  
0                       7.823529  1.0  28.0   17  
1                       2.000000  2.0   2.0    1  
2                       8.500000  3.0  14.0    2  
3                       4.166667  2.0   6.0    6  
4                       1.000000  1.0   1.0    1  
...                          ...  ...   ...  ...  
1083                    6.923077  1.0  14.0   13  
1084                    8.666667  1.0  17.0    3  
1085                    3.000000  1.0   4.0    6  
1086                    8.000000  2.0  14.0    2  
1087                   12.000000  4.0  20.0    5  

[1088 rows x 5 columns]

In [20]:
licitacionPorProvincia = df.groupby(['Provincia']).agg({'tenderPeriod_durationInDays': ['mean', 'min', 'max','size']}).reset_index()
licitacionPorProvincia

Provincia tenderPeriod_durationInDays             \
                                                          mean  min   max   
0                            Azuay                    7.519726  1.0  28.0   
1                          Bolivar                   11.940299  1.0  22.0   
2                           Carchi                    5.455446  1.0  19.0   
3                            Cañar                    6.720000  1.0  31.0   
4                       Chimborazo                    7.308682  1.0  25.0   
5                         Cotopaxi                    7.984456  1.0  20.0   
6                           El Oro                    7.515843  1.0  27.0   
7                       Esmeraldas                    6.861111  1.0  28.0   
8                        Galapagos                   14.993843  1.0  19.0   
9                           Guayas                    7.510730  1.0  57.0   
10                        Imbabura                    6.841155  1.0  22.0   
11                            Loja                    4.903162  1.0  34.0   
12                        Los Rios                    5.384236  1.0  20.0   
13                          Manabi                    7.942928  1.0  30.0   
14                 Morona Santiago                    5.859345  1.0  20.0   
15                            Napo                    5.282430  1.0  33.0   
16                        Orellana                   11.658228  1.0  15.0   
17                         Pastaza                    8.895349  1.0  19.0   
18                       Pichincha                   18.483829  1.0  34.0   
19                     Santa Elena                    6.066667  3.0  15.0   
20  Santo Domingo de los Tsachilas                    5.493506  1.0  22.0   
21                       Sucumbios                   12.690265  3.0  17.0   
22                      Tungurahua                    9.098361  1.0  24.0   
23                Zamora Chinchipe                    7.009009  1.0  24.0   

           
     size  
0     583  
1     134  
2     101  
3     200  
4     311  
5     193  
6     789  
7     108  
8   71953  
9    1398  
10    277  
11    506  
12    203  
13    403  
14    519  
15    609  
16     79  
17    258  
18   5380  
19     30  
20     77  
21    113  
22    366  
23    111

In [22]:
licitacionPorMetodo = df.groupby(['procurementMethodDetails']).agg({'tenderPeriod_durationInDays': ['mean', 'min', 'max','size']}).reset_index()
licitacionPorMetodo

procurementMethodDetails  \
                                                        
0                      Asesoría y Patrocinio Jurídico   
1   Asesoría y Patrocinio Jurídico – Cons. puntual...   
2                           Bienes y Servicios únicos   
3          Comunicación Social – Contratación Directa   
4          Comunicación Social – Proceso de Selección   
5   Contrataciones con empresas públicas internaci...   
6     Contratación Directa por Terminación Unilateral   
7   Contratos entre Entidades Públicas o sus subsi...   
8                                          Cotización   
9                                          Licitación   
10                              Licitación de Seguros   
11             Obra artística, científica o literaria   
12                             Repuestos o Accesorios   
13       Transporte de correo interno o internacional   

   tenderPeriod_durationInDays                     
                          mean   min   max   size  
0                     4.060606   1.0  12.0     66  
1                     5.000000   1.0   7.0      5  
2                     4.062844   1.0  19.0   2180  
3                     4.126280   1.0  14.0   1758  
4                     3.805405   1.0  14.0    370  
5                     4.000000   4.0   4.0      1  
6                     5.472222   3.0  26.0     36  
7                     4.091099   1.0  20.0    955  
8                    14.971067   9.0  31.0  73100  
9                    25.373764  14.0  57.0   3438  
10                   15.854004  14.0  28.0   1411  
11                    2.909457   1.0  13.0    497  
12                    4.629108   1.0  17.0    852  
13                    4.687500   2.0   8.0     32